<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Necessary-Libraries" data-toc-modified-id="Importing-Necessary-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Necessary Libraries</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modelling</a></span></li><li><span><a href="#Forecasting" data-toc-modified-id="Forecasting-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Forecasting</a></span><ul class="toc-item"><li><span><a href="#Exploration" data-toc-modified-id="Exploration-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Exploration</a></span></li></ul></li></ul></div>

# Importing Necessary Libraries

In [1]:
# Importing general libraries
import numpy as np
import pandas as pd

# Importing libraries for preprocessing
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Importing modelling and evaluation tools
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from scipy.stats import randint

# Misc.
import hashlib

2024-08-02 23:19:16.247134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Adjusting display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 200)
sns.set_theme()

# Preprocessing

In [3]:
df=pd.read_csv('Data/Wide Dataset.csv')

In [4]:
df_panel=pd.read_csv('Data/Panel Encoded.csv')

In [5]:
df_panel=df_panel.drop('Company ID', axis=1)

In [6]:
train_list = []
val_list = []
test_list = []

company_names = df_panel['Company Name'].unique()

for company_name in company_names:
    company_data = df_panel[df_panel['Company Name'] == company_name]
    num_rows = company_data.shape[0]
    print(f"Company Name: {company_name}, Total Rows: {num_rows}")
    
    if num_rows > 2:  
        company_data = company_data.sort_values(by='Year')
        train_split_index = int(0.7 * num_rows)
        val_split_index = int(0.85 * num_rows)
        
        train_data = company_data.iloc[:train_split_index]
        val_data = company_data.iloc[train_split_index:val_split_index]
        test_data = company_data.iloc[val_split_index:]
        
        train_list.append(train_data)
        val_list.append(val_data)
        test_list.append(test_data)
    elif num_rows > 1:  
        company_data = company_data.sort_values(by='Year')
        split_index = int(0.8 * num_rows)
        train_data = company_data.iloc[:split_index]
        test_data = company_data.iloc[split_index:]
        train_list.append(train_data)
        test_list.append(test_data)
    else:
        train_list.append(company_data)

train_df = pd.concat(train_list)
val_df = pd.concat(val_list)
test_df = pd.concat(test_list)


Company Name: 164b777e60a3cca16515022bd9b58a9bc7b9b57377c7224a29aadf53b0efec63, Total Rows: 5
Company Name: 7280bb15309b2413c875cb61c2756ebc1b78bb4e9d0ef4c535d5a0c4d7d37db4, Total Rows: 2
Company Name: dcb2d5ccc1a2d370dff76757c90331431354b313bde303dd08f75bc6c1996cd2, Total Rows: 3
Company Name: d908163a6e692d44d6fa427fd7c549ec865ebc68479a20b40211568d0706c795, Total Rows: 3
Company Name: 10c77dcfd3e19c1595d223a591ad1a5bc71fdfa9bd274d40e495748a33c2e973, Total Rows: 2
Company Name: b4cba85f3e75ff2961a054618a090a8f3c92c5a8f0d4d3a4bd7c8107c45222a9, Total Rows: 5
Company Name: c030a47e58fcd8ea5a90a93e242fb8a242f296a39edc8d1a7f4d4f2da0e1aa8c, Total Rows: 5
Company Name: c24480a8f9462b3ea032cc98c5dc4817c6a1379d652102484de97c791bcaa34f, Total Rows: 5
Company Name: f6316083a5d529c4af4737633e1485d697ce0de10931191479142492b6a0955e, Total Rows: 5
Company Name: ed0d9c7761f0803b6adb9a139c2baba10b67a78e837be72f1fa652bd02419cdf, Total Rows: 5
Company Name: 75e1d8cb1afa57c18f6eca99301d7febf5d3fb3ad7ff02

In [7]:
df_panel.reset_index(inplace=True)
df_panel.set_index(['Company Name', 'Year'], inplace=True)

In [8]:
train_df.reset_index(inplace=True)
val_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)
train_df.set_index(['Company Name', 'Year'], inplace=True)
val_df.set_index(['Company Name', 'Year'], inplace=True)
test_df.set_index(['Company Name', 'Year'], inplace=True)

train_df=train_df.drop('index', axis=1)
val_df=val_df.drop('index', axis=1)
test_df=test_df.drop('index', axis=1)

In [9]:
from sklearn.compose import ColumnTransformer

scaler = MinMaxScaler(feature_range=(0, 1))
column_transformer = ColumnTransformer(
    transformers=[
        ('scaler', scaler, ['Employment', 'GVA'])
    ],
    remainder='passthrough'
)


def prepare_lstm_data(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

In [10]:
train_values = train_df['GVA'].values
print(f"Original train_values shape: {train_values.shape}")

train_values_scaled = scaler.fit_transform(train_values.reshape(-1, 1))
print(f"Scaled train_values shape: {train_values_scaled.shape}")


Original train_values shape: (8835,)
Scaled train_values shape: (8835, 1)


# 1 Time Step

In [11]:
time_steps = 1
X_train, y_train = prepare_lstm_data(train_values_scaled, time_steps)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (8834, 1, 1), y_train shape: (8834, 1)


In [12]:
val_values = val_df['GVA'].values
val_values_scaled = scaler.transform(val_values.reshape(-1, 1))
X_val, y_val = prepare_lstm_data(val_values_scaled, time_steps)
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_val shape: (2353, 1, 1), y_val shape: (2353, 1)


In [13]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=False, input_shape=(time_steps, 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1, validation_data=(X_val, y_val))

Epoch 1/30
277/277 [==============================] - 3s 4ms/step - loss: 2.4814e-04 - val_loss: 5.2174e-04
Epoch 2/30
277/277 [==============================] - 1s 2ms/step - loss: 2.3252e-04 - val_loss: 5.3562e-04
Epoch 3/30
277/277 [==============================] - 1s 2ms/step - loss: 2.1564e-04 - val_loss: 5.5072e-04
Epoch 4/30
277/277 [==============================] - 1s 2ms/step - loss: 1.9918e-04 - val_loss: 5.7162e-04
Epoch 5/30
277/277 [==============================] - 1s 2ms/step - loss: 1.9002e-04 - val_loss: 5.9628e-04
Epoch 6/30
277/277 [==============================] - 1s 2ms/step - loss: 2.0029e-04 - val_loss: 6.8314e-04
Epoch 7/30
277/277 [==============================] - 1s 2ms/step - loss: 2.1366e-04 - val_loss: 6.2965e-04
Epoch 8/30
277/277 [==============================] - 1s 2ms/step - loss: 2.0134e-04 - val_loss: 6.3515e-04
Epoch 9/30
277/277 [==============================] - 1s 2ms/step - loss: 1.8170e-04 - val_loss: 6.3858e-04
Epoch 10/30
277/277 [=======

In [14]:
train_predictions = lstm_model.predict(X_train)
train_residuals = (y_train - train_predictions).flatten()

277/277 [==============================] - 1s 1ms/step


In [15]:
X_train_rf = train_df.drop(columns=['GVA']).iloc[time_steps:].values
y_train_rf = train_residuals[:len(X_train_rf)]

print(f"X_train_rf shape: {X_train_rf.shape}")
print(f"y_train_rf shape: {y_train_rf.shape}")

X_train_rf shape: (8834, 5)
y_train_rf shape: (8834,)


In [16]:
rf_model = RandomForestRegressor(random_state=42)

# Defining and setting up a parameter grid for RandomizedSearchCV

param_dist = {
    'n_estimators': randint(50, 200),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, 
                                   n_iter=100, cv=5, scoring='neg_mean_squared_error', 
                                   verbose=2, n_jobs=-1, random_state=42)

# Performing the random search
random_search.fit(X_train_rf, y_train_rf.ravel())

# Finding the best parameters from the search
best_params_random = random_search.best_params_

print(f"Best parameters from RandomizedSearchCV: {best_params_random}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters from RandomizedSearchCV: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 77}


In [17]:
# Defining a narrower parameter grid for grid search based on results from random search
param_grid = {
    'n_estimators': [best_params_random['n_estimators'] - 10, best_params_random['n_estimators'], best_params_random['n_estimators'] + 10],
    'max_features': [best_params_random['max_features']],
    'max_depth': [None if best_params_random['max_depth'] is None else max(1, best_params_random['max_depth'] - 10), 
                  best_params_random['max_depth'], 
                  None if best_params_random['max_depth'] is None else best_params_random['max_depth'] + 10],
    'min_samples_split': [max(2, best_params_random['min_samples_split'] - 1), best_params_random['min_samples_split'], best_params_random['min_samples_split'] + 1],
    'min_samples_leaf': [max(1, best_params_random['min_samples_leaf'] - 1), best_params_random['min_samples_leaf'], best_params_random['min_samples_leaf'] + 1]
}

param_grid['min_samples_split'] = [max(2, x) for x in param_grid['min_samples_split']]
param_grid['min_samples_leaf'] = [max(1, x) for x in param_grid['min_samples_leaf']]

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Performing the grid search
grid_search.fit(X_train_rf, y_train_rf.ravel())

# Getting the best parameters and estimator
best_params_grid = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print(f"Best parameters from GridSearchCV: {best_params_grid}")


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters from GridSearchCV: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 77}


In [18]:
train_residuals_rf = best_rf_model.predict(X_train_rf)
train_final_predictions = train_predictions + train_residuals_rf.reshape(-1, 1)

In [19]:
test_values = test_df['GVA'].values
test_values_scaled = scaler.transform(test_values.reshape(-1, 1))
X_test, y_test = prepare_lstm_data(test_values_scaled, time_steps)


In [20]:
test_predictions_lstm = lstm_model.predict(X_test)

122/122 [==============================] - 0s 1ms/step


In [21]:
X_test_rf = test_df.iloc[time_steps:].drop(columns=['GVA']).values


In [22]:
test_residuals_rf = best_rf_model.predict(X_test_rf)

In [23]:
final_predictions = test_predictions_lstm + test_residuals_rf.reshape(-1, 1)

In [24]:
final_predictions[:5], y_test[:5]

(array([[ 0.00039653],
        [ 0.00055411],
        [-0.00124543],
        [ 0.00047434],
        [ 0.0004473 ]]),
 array([[0.00047587],
        [0.00061402],
        [0.00046504],
        [0.00052218],
        [0.00048461]]))

In [25]:
final_predictions_inverse = scaler.inverse_transform(final_predictions)
test_values_inverse = scaler.inverse_transform(test_values_scaled[:len(final_predictions)])

actual_values = test_values_inverse.flatten()
predicted_values_lstm = scaler.inverse_transform(test_predictions_lstm).flatten()
predicted_values_hybrid = final_predictions_inverse.flatten()

In [26]:
final_prediction_df = pd.DataFrame(final_predictions_inverse, columns =['Predicted GVA']) 

final_prediction_df

,Predicted GVA
0,-5002.684809
1,87992.755036
2,-973949.843485
3,40914.332258
4,24959.664459
...,...
3878,-23140.377009
3879,41713.345006
3880,-5019.277974
3881,-14720.770195


In [27]:
actual_values_df = pd.DataFrame(test_values_inverse, columns =['Actual GVA']) 
comparison = final_prediction_df.join(actual_values_df)

In [28]:
drop_2_cv_rmse_inv = np.sqrt(mean_squared_error(test_values_inverse, final_predictions_inverse))
print(f'Root Mean Squared Error on Test Set (Original Scale): {drop_2_cv_rmse_inv}')

drop_2_cv_mae_inv = mean_absolute_error(test_values_inverse, final_predictions_inverse)
print(f'Mean Absolute Error on Test Set (Original Scale): {drop_2_cv_mae_inv}')

drop_2_cv_r2_inv = r2_score(test_values_inverse, final_predictions_inverse)
print(f'R-squared on Test Set (Original Scale): {drop_2_cv_r2_inv}')

drop_2_cv_mape_inv = mean_absolute_percentage_error(test_values_inverse, final_predictions_inverse)
print(f'Mean Absolute Percentage Error on Train Set (Original Scale): {drop_2_cv_mape_inv}')

Root Mean Squared Error on Test Set (Original Scale): 8752808.326017795
Mean Absolute Error on Test Set (Original Scale): 961373.585372715
R-squared on Test Set (Original Scale): 0.6885651895278562
Mean Absolute Percentage Error on Train Set (Original Scale): 8.380076236049662


In [29]:
y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).reshape(-1)
train_final_predictions_original = scaler.inverse_transform(train_final_predictions).reshape(-1)

drop_2_cv_rmse_train_inv = np.sqrt(mean_squared_error(y_train_original, train_final_predictions_original))
print(f'Root Mean Squared Error on Train Set (Original Scale): {drop_2_cv_rmse_train_inv}')

drop_2_cv_mae_train_inv = mean_absolute_error(y_train_original, train_final_predictions_original)
print(f'Mean Absolute Error on Train Set (Original Scale): {drop_2_cv_mae_train_inv}')

drop_2_cv_r2_train_inv = r2_score(y_train_original, train_final_predictions_original)
print(f'R-squared on Train Set (Original Scale): {drop_2_cv_r2_train_inv}')


Root Mean Squared Error on Train Set (Original Scale): 4718600.107749921
Mean Absolute Error on Train Set (Original Scale): 490134.9938313252
R-squared on Train Set (Original Scale): 0.7686875196524972


# 2 Time Steps

In [30]:
time_steps = 2
X_train, y_train = prepare_lstm_data(train_values_scaled, time_steps)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (8833, 2, 1), y_train shape: (8833, 1)


In [31]:
val_values = val_df['GVA'].values
val_values_scaled = scaler.transform(val_values.reshape(-1, 1))
X_val, y_val = prepare_lstm_data(val_values_scaled, time_steps)
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_val shape: (2352, 2, 1), y_val shape: (2352, 1)


In [32]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=False, input_shape=(time_steps, 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1, validation_data=(X_val, y_val))

Epoch 1/30
277/277 [==============================] - 3s 4ms/step - loss: 2.4249e-04 - val_loss: 5.2838e-04
Epoch 2/30
277/277 [==============================] - 1s 3ms/step - loss: 2.4432e-04 - val_loss: 5.2668e-04
Epoch 3/30
277/277 [==============================] - 1s 3ms/step - loss: 2.2851e-04 - val_loss: 5.4011e-04
Epoch 4/30
277/277 [==============================] - 1s 3ms/step - loss: 2.3272e-04 - val_loss: 5.4742e-04
Epoch 5/30
277/277 [==============================] - 1s 3ms/step - loss: 2.1902e-04 - val_loss: 5.5001e-04
Epoch 6/30
277/277 [==============================] - 1s 2ms/step - loss: 2.2918e-04 - val_loss: 5.5008e-04
Epoch 7/30
277/277 [==============================] - 1s 3ms/step - loss: 2.0230e-04 - val_loss: 5.5979e-04
Epoch 8/30
277/277 [==============================] - 1s 2ms/step - loss: 2.0512e-04 - val_loss: 5.6590e-04
Epoch 9/30
277/277 [==============================] - 1s 2ms/step - loss: 1.9388e-04 - val_loss: 5.7661e-04
Epoch 10/30
277/277 [=======

In [33]:
train_predictions = lstm_model.predict(X_train)
train_residuals = (y_train - train_predictions).flatten()

277/277 [==============================] - 1s 972us/step


In [34]:
X_train_rf = train_df.drop(columns=['GVA']).iloc[time_steps:].values
y_train_rf = train_residuals[:len(X_train_rf)]

print(f"X_train_rf shape: {X_train_rf.shape}")
print(f"y_train_rf shape: {y_train_rf.shape}")

X_train_rf shape: (8833, 5)
y_train_rf shape: (8833,)


In [35]:
rf_model = RandomForestRegressor(random_state=42)

# Defining and setting up a parameter grid for RandomizedSearchCV

param_dist = {
    'n_estimators': randint(50, 200),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, 
                                   n_iter=100, cv=5, scoring='neg_mean_squared_error', 
                                   verbose=2, n_jobs=-1, random_state=42)

# Performing the random search
random_search.fit(X_train_rf, y_train_rf.ravel())

# Finding the best parameters from the search
best_params_random = random_search.best_params_

print(f"Best parameters from RandomizedSearchCV: {best_params_random}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters from RandomizedSearchCV: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 177}


In [36]:
# Defining a narrower parameter grid for grid search based on results from random search
param_grid = {
    'n_estimators': [best_params_random['n_estimators'] - 10, best_params_random['n_estimators'], best_params_random['n_estimators'] + 10],
    'max_features': [best_params_random['max_features']],
    'max_depth': [None if best_params_random['max_depth'] is None else max(1, best_params_random['max_depth'] - 10), 
                  best_params_random['max_depth'], 
                  None if best_params_random['max_depth'] is None else best_params_random['max_depth'] + 10],
    'min_samples_split': [max(2, best_params_random['min_samples_split'] - 1), best_params_random['min_samples_split'], best_params_random['min_samples_split'] + 1],
    'min_samples_leaf': [max(1, best_params_random['min_samples_leaf'] - 1), best_params_random['min_samples_leaf'], best_params_random['min_samples_leaf'] + 1]
}

param_grid['min_samples_split'] = [max(2, x) for x in param_grid['min_samples_split']]
param_grid['min_samples_leaf'] = [max(1, x) for x in param_grid['min_samples_leaf']]

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Performing the grid search
grid_search.fit(X_train_rf, y_train_rf.ravel())

# Getting the best parameters and estimator
best_params_grid = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print(f"Best parameters from GridSearchCV: {best_params_grid}")


Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END max_depth=20, max_features=log2, min_samples_leaf=3, min_samples_split=8, n_estimators=124; total time=   0.9s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=153; total time=   1.2s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=107; total time=   1.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=98; total time=   0.5s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=4, n_estimators=157; total time=   1.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=3, min_samples_split=6, n_estimators=122; total time=   0.9s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=109; total time=   0.9s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=8, n_estimato

In [37]:
train_residuals_rf = best_rf_model.predict(X_train_rf)
train_final_predictions = train_predictions + train_residuals_rf.reshape(-1, 1)

In [38]:
test_values = test_df['GVA'].values
test_values_scaled = scaler.transform(test_values.reshape(-1, 1))
X_test, y_test = prepare_lstm_data(test_values_scaled, time_steps)


In [39]:
test_predictions_lstm = lstm_model.predict(X_test)

122/122 [==============================] - 0s 2ms/step


In [40]:
X_test_rf = test_df.iloc[time_steps:].drop(columns=['GVA']).values


In [41]:
test_residuals_rf = best_rf_model.predict(X_test_rf)

In [42]:
final_predictions = test_predictions_lstm + test_residuals_rf.reshape(-1, 1)

In [43]:
final_predictions[:5], y_test[:5]

(array([[5.09659860e-04],
        [8.77942750e-05],
        [4.32182301e-04],
        [4.85761664e-04],
        [4.90729111e-04]]),
 array([[0.00061402],
        [0.00046504],
        [0.00052218],
        [0.00048461],
        [0.00050329]]))

In [44]:
final_predictions_inverse = scaler.inverse_transform(final_predictions)
test_values_inverse = scaler.inverse_transform(test_values_scaled[:len(final_predictions)])

actual_values = test_values_inverse.flatten()
predicted_values_lstm = scaler.inverse_transform(test_predictions_lstm).flatten()
predicted_values_hybrid = final_predictions_inverse.flatten()

In [45]:
final_prediction_df = pd.DataFrame(final_predictions_inverse, columns =['Predicted GVA']) 

final_prediction_df

,Predicted GVA
0,61759.436983
1,-187191.021528
2,16038.537569
3,47656.682414
4,50588.061760
...,...
3877,26111.424062
3878,36272.205990
3879,395788.920624
3880,17695.879679


In [46]:
actual_values_df = pd.DataFrame(test_values_inverse, columns =['Actual GVA']) 
comparison = final_prediction_df.join(actual_values_df)

In [47]:
drop_2_cv_rmse_inv = np.sqrt(mean_squared_error(test_values_inverse, final_predictions_inverse))
print(f'Root Mean Squared Error on Test Set (Original Scale): {drop_2_cv_rmse_inv}')

drop_2_cv_mae_inv = mean_absolute_error(test_values_inverse, final_predictions_inverse)
print(f'Mean Absolute Error on Test Set (Original Scale): {drop_2_cv_mae_inv}')

drop_2_cv_r2_inv = r2_score(test_values_inverse, final_predictions_inverse)
print(f'R-squared on Test Set (Original Scale): {drop_2_cv_r2_inv}')

drop_2_cv_mape_inv = mean_absolute_percentage_error(test_values_inverse, final_predictions_inverse)
print(f'Mean Absolute Percentage Error on Train Set (Original Scale): {drop_2_cv_mape_inv}')

Root Mean Squared Error on Test Set (Original Scale): 25591790.775099747
Mean Absolute Error on Test Set (Original Scale): 2527317.475829604
R-squared on Test Set (Original Scale): -1.6617183085201668
Mean Absolute Percentage Error on Train Set (Original Scale): 20.890448663724477


In [48]:
y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).reshape(-1)
train_final_predictions_original = scaler.inverse_transform(train_final_predictions).reshape(-1)

drop_2_cv_rmse_train_inv = np.sqrt(mean_squared_error(y_train_original, train_final_predictions_original))
print(f'Root Mean Squared Error on Train Set (Original Scale): {drop_2_cv_rmse_train_inv}')

drop_2_cv_mae_train_inv = mean_absolute_error(y_train_original, train_final_predictions_original)
print(f'Mean Absolute Error on Train Set (Original Scale): {drop_2_cv_mae_train_inv}')

drop_2_cv_r2_train_inv = r2_score(y_train_original, train_final_predictions_original)
print(f'R-squared on Train Set (Original Scale): {drop_2_cv_r2_train_inv}')


Root Mean Squared Error on Train Set (Original Scale): 3241613.7488702983
Mean Absolute Error on Train Set (Original Scale): 498444.6128345524
R-squared on Train Set (Original Scale): 0.8908443192726891


# 3 Time Steps

In [49]:
time_steps = 3
X_train, y_train = prepare_lstm_data(train_values_scaled, time_steps)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (8832, 3, 1), y_train shape: (8832, 1)


In [50]:
val_values = val_df['GVA'].values
val_values_scaled = scaler.transform(val_values.reshape(-1, 1))
X_val, y_val = prepare_lstm_data(val_values_scaled, time_steps)
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_val shape: (2351, 3, 1), y_val shape: (2351, 1)


In [51]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=False, input_shape=(time_steps, 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1, validation_data=(X_val, y_val))

Epoch 1/30
276/276 [==============================] - 3s 5ms/step - loss: 2.9705e-04 - val_loss: 5.1498e-04
Epoch 2/30
276/276 [==============================] - 1s 3ms/step - loss: 2.7277e-04 - val_loss: 5.1773e-04
Epoch 3/30
276/276 [==============================] - 1s 2ms/step - loss: 2.5485e-04 - val_loss: 5.1794e-04
Epoch 4/30
276/276 [==============================] - 1s 2ms/step - loss: 2.4262e-04 - val_loss: 5.2991e-04
Epoch 5/30
276/276 [==============================] - 1s 2ms/step - loss: 2.3431e-04 - val_loss: 5.3317e-04
Epoch 6/30
276/276 [==============================] - 1s 2ms/step - loss: 2.1421e-04 - val_loss: 5.4170e-04
Epoch 7/30
276/276 [==============================] - 1s 2ms/step - loss: 2.0189e-04 - val_loss: 5.5795e-04
Epoch 8/30
276/276 [==============================] - 1s 2ms/step - loss: 2.0539e-04 - val_loss: 5.7339e-04
Epoch 9/30
276/276 [==============================] - 1s 2ms/step - loss: 1.9438e-04 - val_loss: 5.8973e-04
Epoch 10/30
276/276 [=======

In [52]:
train_predictions = lstm_model.predict(X_train)
train_residuals = (y_train - train_predictions).flatten()

276/276 [==============================] - 1s 1ms/step


In [53]:
X_train_rf = train_df.drop(columns=['GVA']).iloc[time_steps:].values
y_train_rf = train_residuals[:len(X_train_rf)]

print(f"X_train_rf shape: {X_train_rf.shape}")
print(f"y_train_rf shape: {y_train_rf.shape}")

X_train_rf shape: (8832, 5)
y_train_rf shape: (8832,)


In [54]:
rf_model = RandomForestRegressor(random_state=42)

# Defining and setting up a parameter grid for RandomizedSearchCV

param_dist = {
    'n_estimators': randint(50, 200),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, 
                                   n_iter=100, cv=5, scoring='neg_mean_squared_error', 
                                   verbose=2, n_jobs=-1, random_state=42)

# Performing the random search
random_search.fit(X_train_rf, y_train_rf.ravel())

# Finding the best parameters from the search
best_params_random = random_search.best_params_

print(f"Best parameters from RandomizedSearchCV: {best_params_random}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters from RandomizedSearchCV: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 117}


In [55]:
# Defining a narrower parameter grid for grid search based on results from random search
param_grid = {
    'n_estimators': [best_params_random['n_estimators'] - 10, best_params_random['n_estimators'], best_params_random['n_estimators'] + 10],
    'max_features': [best_params_random['max_features']],
    'max_depth': [None if best_params_random['max_depth'] is None else max(1, best_params_random['max_depth'] - 10), 
                  best_params_random['max_depth'], 
                  None if best_params_random['max_depth'] is None else best_params_random['max_depth'] + 10],
    'min_samples_split': [max(2, best_params_random['min_samples_split'] - 1), best_params_random['min_samples_split'], best_params_random['min_samples_split'] + 1],
    'min_samples_leaf': [max(1, best_params_random['min_samples_leaf'] - 1), best_params_random['min_samples_leaf'], best_params_random['min_samples_leaf'] + 1]
}

param_grid['min_samples_split'] = [max(2, x) for x in param_grid['min_samples_split']]
param_grid['min_samples_leaf'] = [max(1, x) for x in param_grid['min_samples_leaf']]

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Performing the grid search
grid_search.fit(X_train_rf, y_train_rf.ravel())

# Getting the best parameters and estimator
best_params_grid = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print(f"Best parameters from GridSearchCV: {best_params_grid}")


Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END max_depth=30, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=183; total time=   2.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=8, n_estimators=141; total time=   1.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=165; total time=   1.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=108; total time=   0.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=108; total time=   0.9s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=9, n_estimators=162; total time=   1.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=162; total time=   1.2s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=7, n_estimator

In [56]:
train_residuals_rf = best_rf_model.predict(X_train_rf)
train_final_predictions = train_predictions + train_residuals_rf.reshape(-1, 1)

In [57]:
test_values = test_df['GVA'].values
test_values_scaled = scaler.transform(test_values.reshape(-1, 1))
X_test, y_test = prepare_lstm_data(test_values_scaled, time_steps)


In [58]:
test_predictions_lstm = lstm_model.predict(X_test)

122/122 [==============================] - 1s 2ms/step


In [59]:
X_test_rf = test_df.iloc[time_steps:].drop(columns=['GVA']).values


In [60]:
test_residuals_rf = best_rf_model.predict(X_test_rf)

In [61]:
final_predictions = test_predictions_lstm + test_residuals_rf.reshape(-1, 1)

In [62]:
final_predictions[:5], y_test[:5]

(array([[-0.00147314],
        [ 0.00066616],
        [ 0.00045281],
        [ 0.00058158],
        [ 0.00065692]]),
 array([[0.00046504],
        [0.00052218],
        [0.00048461],
        [0.00050329],
        [0.00079815]]))

In [63]:
final_predictions_inverse = scaler.inverse_transform(final_predictions)
test_values_inverse = scaler.inverse_transform(test_values_scaled[:len(final_predictions)])

actual_values = test_values_inverse.flatten()
predicted_values_lstm = scaler.inverse_transform(test_predictions_lstm).flatten()
predicted_values_hybrid = final_predictions_inverse.flatten()

In [64]:
final_prediction_df = pd.DataFrame(final_predictions_inverse, columns =['Predicted GVA']) 

final_prediction_df

,Predicted GVA
0,-1.108324e+06
1,1.541135e+05
2,2.821265e+04
3,1.041998e+05
4,1.486610e+05
...,...
3876,3.412817e+04
3877,4.790842e+04
3878,2.700279e+05
3879,2.697712e+04


In [65]:
actual_values_df = pd.DataFrame(test_values_inverse, columns =['Actual GVA']) 
comparison = final_prediction_df.join(actual_values_df)

In [66]:
drop_2_cv_rmse_inv = np.sqrt(mean_squared_error(test_values_inverse, final_predictions_inverse))
print(f'Root Mean Squared Error on Test Set (Original Scale): {drop_2_cv_rmse_inv}')

drop_2_cv_mae_inv = mean_absolute_error(test_values_inverse, final_predictions_inverse)
print(f'Mean Absolute Error on Test Set (Original Scale): {drop_2_cv_mae_inv}')

drop_2_cv_r2_inv = r2_score(test_values_inverse, final_predictions_inverse)
print(f'R-squared on Test Set (Original Scale): {drop_2_cv_r2_inv}')

drop_2_cv_mape_inv = mean_absolute_percentage_error(test_values_inverse, final_predictions_inverse)
print(f'Mean Absolute Percentage Error on Train Set (Original Scale): {drop_2_cv_mape_inv}')

Root Mean Squared Error on Test Set (Original Scale): 26538238.326063197
Mean Absolute Error on Test Set (Original Scale): 2667752.425138022
R-squared on Test Set (Original Scale): -1.8614972616122047
Mean Absolute Percentage Error on Train Set (Original Scale): 24.31186571123505


In [67]:
y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).reshape(-1)
train_final_predictions_original = scaler.inverse_transform(train_final_predictions).reshape(-1)

drop_2_cv_rmse_train_inv = np.sqrt(mean_squared_error(y_train_original, train_final_predictions_original))
print(f'Root Mean Squared Error on Train Set (Original Scale): {drop_2_cv_rmse_train_inv}')

drop_2_cv_mae_train_inv = mean_absolute_error(y_train_original, train_final_predictions_original)
print(f'Mean Absolute Error on Train Set (Original Scale): {drop_2_cv_mae_train_inv}')

drop_2_cv_r2_train_inv = r2_score(y_train_original, train_final_predictions_original)
print(f'R-squared on Train Set (Original Scale): {drop_2_cv_r2_train_inv}')


Root Mean Squared Error on Train Set (Original Scale): 3693185.9593568696
Mean Absolute Error on Train Set (Original Scale): 636197.8920940919
R-squared on Train Set (Original Scale): 0.8583302229799694
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=4, n_estimators=168; total time=   2.9s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=3, n_estimators=70; total time=   1.0s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=110; total time=   0.9s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=4, n_estimators=98; total time=   1.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=6, n_estimators=144; total time=   1.6s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=116; total time=   1.3s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_s